In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import tensorflow as tf

print(f"Tf version: {tf.__version__}")

Tf version: 2.0.0


In [2]:
from src.models import utils

2.0.0


In [3]:
utils.foo('foo')

'bar'

In [4]:
utils.foo('as')

'baz'

In [5]:
import tensorflow_datasets as tfds

In [6]:
dataset, info = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00',
                          with_info=True)

train_dataset = dataset['train']

In [9]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [12]:
BUFFER_SIZE = 30000
BATCH_SIZE = 128

In [13]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [18]:
for reviews in train_dataset.take(3):
    review_text = reviews['data']
    print(f"Review Text: {review_text.get('review_body').numpy()}")
    print(f"Star Rating: {review_text.get('star_rating').numpy()}")
    print(tf.where(review_text.get('star_rating') > 3, 1, 0).numpy())
    print("\n")

Review Text: b'great tool fun easy to use great price lots of features can use as webcam with face recognition, still image mode, lcd screen on board most pro recorders dont have it... records in HD can do 1080 but at lower fr BUT who records in 1080 untill u have pro editing software and shoot on red? 720 is great at 30fps accurate with sound cool suction mount  even though the attachment it self were u clip camera on mount kinda gay but works fine, great buy for great price.'
Star Rating: 5
1


Review Text: b'nice product'
Star Rating: 5
1


Review Text: b'It is ok<br />The box cant handle the bass'
Star Rating: 3
0




In [19]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for _, reviews in train_dataset.enumerate():
    review_text = reviews['data']
    reviews_tokens = tokenizer.tokenize(review_text.get('review_body').numpy())
    vocabulary_set.update(reviews_tokens)

vocab_size = len(vocabulary_set)
print(vocab_size)

73738


In [20]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [25]:
encoder

<TokenTextEncoder vocab_size=73740>

In [30]:
for reviews in train_dataset.take(2):
    review_text = reviews["data"]
    print(review_text.get('review_body').numpy())
    encoded_example = encoder.encode(review_text.get("review_body").numpy())
    print(encoded_example)
    print("\n")

b'great tool fun easy to use great price lots of features can use as webcam with face recognition, still image mode, lcd screen on board most pro recorders dont have it... records in HD can do 1080 but at lower fr BUT who records in 1080 untill u have pro editing software and shoot on red? 720 is great at 30fps accurate with sound cool suction mount  even though the attachment it self were u clip camera on mount kinda gay but works fine, great buy for great price.'
[37278, 42766, 9067, 23279, 58859, 2255, 37278, 25712, 19538, 22866, 73485, 9885, 2255, 7077, 43393, 70564, 56186, 5709, 33666, 50146, 37351, 26089, 51676, 50949, 25780, 33203, 50285, 73577, 55784, 6536, 59122, 66663, 39181, 4782, 9885, 38775, 73471, 67535, 4194, 54732, 60899, 72358, 37953, 66663, 39181, 73471, 58066, 65761, 6536, 50285, 64688, 49159, 7722, 12384, 50949, 26491, 27103, 48199, 37278, 4194, 53546, 72361, 70564, 50833, 55812, 47310, 9635, 15522, 3224, 60289, 24941, 59122, 2286, 69452, 65761, 38582, 72563, 50949,

In [32]:
for index in encoded_example:
    print(f"{index} --------> {encoder.decode([index])}")

47582 --------> nice
64476 --------> product
